In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time
import pickle

In [ ]:
# 웹 드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
driver.get("https://www.siksinhot.com/search?keywords=%EC%84%9C%EC%9A%B8")

In [ ]:
search_list = [
    "강동구","강서구","강남구","도봉구","양천구","마포구","서대문구","은평구","서초구","구로구","중구",
    "동대문구","중랑구","광진구","송파구","강북구","관악구"
]

In [ ]:
driver.find_element(By.CSS_SELECTOR,"#header > header > div.searchBox > div").click()

In [ ]:
driver.find_element(By.CSS_SELECTOR,"#searchInput").send_keys("강북구")
driver.find_element(By.CSS_SELECTOR,"#searchInput").send_keys(Keys.RETURN)

In [ ]:
all_pages = []
for search in search_list:
    input_box = driver.find_element(By.CSS_SELECTOR,"#header > header > div.searchBox > div")
    input_box.click()
    time.sleep(4)
    
    search_input = driver.find_element(By.CSS_SELECTOR,"#searchInput")
    search_input.send_keys(search)
    search_input.send_keys(Keys.RETURN)
    time.sleep(4)
    try:
        while True:
            more_box = driver.find_element(By.CSS_SELECTOR, "#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > a")

            more_box.click()
            time.sleep(2)  # 클릭 후 로딩을 위한 대기 시간 (필요에 따라 조절)

    except:
        page_source = driver.page_source
        all_pages.append(page_source)
        pass 
    
    

In [ ]:
len(all_pages)

In [ ]:
for page in all_pages:
    soup = bs(page,"lxml")
    soup.select()

In [5]:
url = "https://www.siksinhot.com/search?keywords=%EC%84%9C%EB%8C%80%EB%AC%B8%EA%B5%AC"
r = requests.get(url)
soup = bs(r.text,"lxml")


In [6]:
main = soup.select("#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > ul > li")

In [7]:
page_list = []

for page in all_pages:
    soup = bs(page,"lxml")
    main = soup.select("#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > ul > li")
    for i in main:
        dict_list = {}
        title = i.select_one("h2").text
        location = i.select_one(".cate>a:first-child").text
        division = i.select_one(".cate>a:last-child").text
        view = i.select_one(".counting").text
        score = i.select_one(".score").text
        book_mark = i.select_one(".counting>span:nth-of-type(2)").text
        url = i.select_one("figure > a").get('href')
        address = i.select_one("figure > a > img").get('alt').split(",")[-1] if i.select_one("figure > a > img") is not None else "None"
        
    
        dict_list["제목"] = title 
        dict_list["지역"] = location
        dict_list["분류"] = division
        dict_list["조회수"] = view
        dict_list["평점"] = score
        dict_list["북마크"] = book_mark
        dict_list["url"] = url
        dict_list["주소"] = address
        
        page_list.append(dict_list)

NameError: name 'all_pages' is not defined

In [8]:
page_list

[]

In [ ]:
data = pd.DataFrame(page_list)

In [9]:
data.to_csv("식신.csv",encoding="utf-8",index=False)

NameError: name 'data' is not defined

In [11]:
data = pd.read_csv("식신.csv")

In [12]:
data[data["제목"].str.contains("화원")]

,제목,지역,분류,조회수,평점,북마크,url,주소
1077,화원,건대/자양/화양,중식,73054,2.8,4.0,https://www.siksinhot.com/P/364544,서울특별시 광진구 능동로16길 56-6


In [13]:
data

,제목,지역,분류,조회수,평점,북마크,url,주소
0,동구밖오리구이,천호/강동/둔촌,오리고기,869018,3.7,18.0,https://www.siksinhot.com/P/342684,서울특별시 강동구 진황도로 227 하선빌딩
1,양철북(강동구청점),천호/강동/둔촌,곱창/대창,154918,3.3,8.0,https://www.siksinhot.com/P/252209,양철북(강동구청점)
2,강동호치민,천호/강동/둔촌,인도네시아,46365,2.8,5.0,https://www.siksinhot.com/P/480260,서울특별시 강동구 천호대로 1089 복합시설동
3,강동KD부대찌개,천호/강동/둔촌,부대찌개,52013,2.8,3.0,https://www.siksinhot.com/P/373435,서울특별시 강동구 올림픽로62길 7
4,떡갈비연구소 강동점,천호/강동/둔촌,한정식,1475,평가중,NaN,https://www.siksinhot.com/P/1303788,서울특별시 강동구 양재대로111길 54
...,...,...,...,...,...,...,...,...
2023,왕건이 고기집,사당/이수,소구이/불고기,3259,평가중,NaN,https://www.siksinhot.com/P/335346,서울 관악구 남현3길 78
2024,우리설렁탕,봉천/신림,설렁탕/곰탕/갈비탕,59607,3.4,7.0,https://www.siksinhot.com/P/321131,서울특별시 관악구 신림로 190
2025,왕돈까스 왕냉면,봉천/신림,경양식/돈가스,70072,평가중,2.0,https://www.siksinhot.com/P/313642,서울특별시 관악구 남부순환로 1377
2026,맛치킨호프,사당/이수,치킨/통닭,2391,평가중,NaN,https://www.siksinhot.com/P/310395,서울특별시 관악구 남현3길 70


In [14]:
detail_pages = [] 
for links in data["url"]:
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
    driver.get(links)
    time.sleep(3)
    
    try:
        while True:
            more_box2 = driver.find_element(By.CSS_SELECTOR, "#storeView > div.container > div.store_review > a")

            more_box2.click()
            time.sleep(2)  # 클릭 후 로딩을 위한 대기 시간 (필요에 따라 조절)

    except: 
        page_source2 = driver.page_source
        detail_pages.append(page_source2)
        driver.quit()
    

KeyboardInterrupt: 

In [15]:
len(detail_pages)

716

In [16]:
# 이거 진짜 신중하게 써라ㅠㅠㅠㅠㅠ덮어지뮤ㅠㅠㅠㅠㅠㅠ으어 내 이틀
with open("식신디테일정보.pkl","wb") as file:
    pickle.dump(detail_pages,file)

In [3]:
with open("식신디테일정보.pkl","rb") as read_file:
    all_pages = pickle.load(read_file)

EOFError: Ran out of input

In [ ]:
pwd

In [ ]:
len(all_pages)

In [ ]:
all_pages[-1]

In [ ]:
for k in bs(all_pages[-1],"lxml").select(".info_text"):
    print(k.text)

In [ ]:
for i in all_pages[1]:
    soup = bs(i,"lxml")
    for j in soup.select(".info_text"):
        print(j.select_one(".inline-div"))
    

In [ ]:
len(detail_pages)

In [ ]:
len(all_pages)

In [10]:
all_pages.extend(detail_pages)

NameError: name 'all_pages' is not defined

In [ ]:
len(all_pages)

In [4]:
driver.find_element(By.CSS_SELECTOR,"#storeView > div.container > div.store_review > a").click()

NameError: name 'driver' is not defined